Let's try some basic tests of `VB.jl`:

In [1]:
include("VB.jl")
using VB
using Distributions

Let's try to make a Gaussian node with several different sizes and parameterizations:

In [2]:
ndims = (5, 5)  # node dimensions
vardims = 3  # dimension of variable

3

In [3]:
μ = Array{Vector{Float64}}(ndims)
Σ = Array{Matrix{Float64}}(ndims)

for idx in eachindex(μ)
    μ[idx] = rand(vardims)
end

for idx in eachindex(Σ)
    C = rand(vardims, vardims)
    Σ[idx] = C * C'
end

In [4]:
μ[1]

3-element Array{Float64,1}:
 0.83747 
 0.46995 
 0.686269

In [5]:
Σ[1]

3x3 Array{Float64,2}:
 1.31002   0.882374  1.05186 
 0.882374  0.803627  0.858332
 1.05186   0.858332  0.96975 

In [6]:
det(Σ[2])

0.014012860947621825

In [7]:
pars = (μ, Σ)
aa = map(MultivariateNormal, pars...);

In [8]:
rr = RandomNode(aa, Dict{Factor, Symbol}());

In [9]:
typeof(aa)

Array{Distributions.MvNormal{PDMats.PDMat,Array{Float64,1}},2}

In [10]:
typeof(aa) <: AbstractArray{typeof(aa[1]), 2}

true

In [11]:
ss = RandomNode(MultivariateNormal, pars...);

In [12]:
E(ss)

5x5 Array{Array{Float64,1},2}:
 [0.8374697546443692,0.4699503976380863,0.6862686254399861]    …  [0.6701571375371376,0.2727827252436228,0.6972640349466082]  
 [0.6105544249506991,0.25707085004064023,0.3117256038822125]      [0.669778919191172,0.5462262991069637,0.591383545681128]    
 [0.839448192682124,0.5893122654111265,0.10826080911230251]       [0.2335471392073163,0.827141037840821,0.8548403767573183]   
 [0.18124070071910303,0.2188128607196349,0.9388624459689845]      [0.5531960244258731,0.03327988531334092,0.43849622959790846]
 [0.04065274921084261,0.8129384972043578,0.05392478595882122]     [0.9107168155186103,0.268730669897596,0.006926652906375619] 

In [13]:
typeof(ss).parameters[1]

Distributions.MvNormal{PDMats.PDMat,Array{Float64,1}}

# Make a factor and check its value

In [14]:
μ = rand(10)
τ = rand(10)
x = RandomNode(Normal, rand(10), rand(10))
f = LogNormalFactor(x, μ, τ)

VB.LogNormalFactor(VB.RandomNode{Distributions.Normal,1}([Distributions.Normal(μ=0.5768655548197468, σ=0.4637209586860651),Distributions.Normal(μ=0.08826648224037292, σ=0.8927779042993595),Distributions.Normal(μ=0.8030736627243291, σ=0.5721874906817204),Distributions.Normal(μ=0.15928670574916048, σ=0.18963004494103775),Distributions.Normal(μ=0.1390278916551868, σ=0.059081698497177815),Distributions.Normal(μ=0.5742430168391397, σ=0.20311847303268937),Distributions.Normal(μ=0.23189709030724837, σ=0.8385544068074888),Distributions.Normal(μ=0.597365908599919, σ=0.89069025593726),Distributions.Normal(μ=0.4125069579334055, σ=0.9600758170143087),Distributions.Normal(μ=0.6289884559362433, σ=0.6567286254894267)],Dict{VB.Factor,Symbol}()),VB.ConstantNode{Float64,1}([0.8829454447584395,0.12597182089718695,0.8443555517614145,0.8919572544311152,0.3784100880232084,0.6456179635872199,0.6010315534708042,0.6438729791615216,0.3056312389704643,0.02048756743332003],Dict{VB.Factor,Symbol}()),VB.ConstantNod

In [15]:
value(f)

-7.503230381747649

In [16]:
register(f)
x.factormap

Dict{VB.Factor,Symbol} with 1 entry:
  VB.LogNormalFactor(VB.R… => :x

In [17]:
g = EntropyFactor(x)
value(g)

5.91066509603152

In [18]:
naturals(x)

10-element Array{Tuple{Float64,Float64},1}:
 (2.6826323503294716,-2.32517986896246)   
 (0.11074113037722648,-0.6273113392898629)
 (2.452893347545838,-1.5271907555931403)  
 (4.4296073860012966,-13.904510628077455) 
 (39.82868728330323,-143.23991685814116)  
 (13.918642385562631,-12.119122024483932) 
 (0.32978645036769094,-0.7110620705303927)
 (0.752986225373693,-0.6302554385288829)  
 (0.4475280048888758,-0.5424490378670461) 
 (1.4583802735074198,-1.1593060729044975) 

# Let's make a simple model

We want a simple model of inference for a normal distribution:

$$
\begin{aligned}
    y \sim \mathcal{N}(\mu, \tau) \\
    \mu \sim \mathcal{N}(\mu_0, \tau_0) \\
    \tau \sim \mathrm{Ga}(\alpha, \beta) \\
    q(\mu) = \mathcal{N}(m, t) \\
    q(\tau) = \mathrm{Ga}(a, b)
\end{aligned}
$$

This model is conjugate, and can be solved by straightforward updates of the natural parameters of the posterior.

## Make the nodes

We need to make nodes for all the random variables that will need to be updated (we can also create nodes for parameter arrays, but these will be converted automatically if we don't).

In [19]:
dims = (5, 6)

# note: it won't matter much how we initialize here
μ = RandomNode(Normal, zeros(dims), ones(dims))
τ = RandomNode(Gamma, ones(dims), ones(dims))

y = rand(dims)

5x6 Array{Float64,2}:
 0.0570032  0.418675  0.0178485  0.0017706  0.795433  0.140515
 0.277428   0.456211  0.304502   0.843995   0.867431  0.763381
 0.243259   0.807357  0.803148   0.365147   0.633817  0.8252  
 0.52402    0.34655   0.914809   0.928525   0.319981  0.326039
 0.673777   0.369792  0.739706   0.75043    0.584376  0.108832

Now make factors: We need a Normal factor for the observation model plus a prior and an entropy for each node.

In [20]:
obs = LogNormalFactor(y, μ, τ)
μ_prior = LogNormalFactor(μ, zeros(dims), 2 * ones(dims))
τ_prior = LogGammaFactor(τ, 1.1 * ones(dims), ones(dims))

VB.LogGammaFactor(VB.RandomNode{Distributions.Gamma,2}(5x6 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=1.0, θ=1.0)  …  Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0),Dict{VB.Factor,Symbol}()),VB.ConstantNode{Float64,2}(5x6 Array{Float64,2}:
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1,Dict{VB.Factor,Symbol}()),VB.ConstantNode{Float64,2}(5x6 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0,Dict{VB.Factor,Symbol}()))

In [21]:
value(obs), value(μ_prior), value(τ_prior)

(-39.00266898410522,-67.96536370453937,7.496173237795199)

In [22]:
m = VBModel(Dict(:μ => μ, :τ => τ), [obs, μ_prior, τ_prior])

VB.VBModel(Dict{Symbol,VB.Node}(:τ=>VB.RandomNode{Distributions.Gamma,2}(5x6 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=1.0, θ=1.0)  …  Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0),Dict{VB.Factor,Symbol}(VB.LogGammaFactor(VB.RandomNode{Distributions.Gamma,2}(#= circular reference =#),VB.ConstantNode{Float64,2}(5x6 Array{Float64,2}:
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1,Dict{VB.Factor,Symbol}()),VB.ConstantNode{Float64,2}(5x6 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0,

In [23]:
m.nodes[:μ].factormap

Dict{VB.Factor,Symbol} with 2 entries:
  VB.LogNormalFactor(VB.R… => :x
  VB.LogNormalFactor(VB.C… => :μ